In [28]:
using Combinatorics
using BenchmarkTools
using Polynomials
using Plots
using Jacobi
using SparseArrays

* $d$ : dimension
* $N$ : number of particle 
* $M$ : degree of polynomial

* **basis function** : $\prod\limits_{i=1}^N \sum\limits_{z=1}^{N} \prod\limits_{j=1}^d T_{k_{ij}}(r_{zj})* \prod\limits_{i=1}^N \prod\limits_{j=1}^d (r_c^2-r_{ij}^2)$
* boundary_func : $\prod\limits_{i=1}^N \prod\limits_{j=1}^d (r_c^2-r_{ij}^2)$
* Prod_Tk : $\prod\limits_{j=1}^d T_{k_{ij}}(r_{zj})$
* Sum_Tk : $\sum\limits_{z=1}^{N} \prod\limits_{j=1}^d T_{k_{ij}}(r_{zj})$
* basis_func : $\prod\limits_{i=1}^N \sum\limits_{z=1}^{N} \prod\limits_{j=1}^d T_{k_{ij}}(r_{zj})* \prod\limits_{i=1}^N \prod\limits_{j=1}^d (r_c^2-r_{ij}^2)$

In [29]:
function basis_function(N,d,r,k,rc)
    Prod_Tk=zeros(N,N)
    Sum_Tk=zeros(1,N)
        for i=1:N 
            for z=1:N
                Prod_temp=1
                for j=1:d
                    Prod_temp=Prod_temp*chebyshev(r[z,j],k[i,j])
                end
                Prod_Tk[i,z]=Prod_temp
            end
            Sum_Tk[i]=sum(Prod_Tk[i,:])
        end
    boundary_func=vcat(map(r->rc^2-r^2, r))
    basis_func=prod(Sum_Tk)*prod(boundary_func)
    return basis_func,Sum_Tk,Prod_Tk,boundary_func
end

basis_function (generic function with 1 method)


* $\dfrac{d^2 T_k}{dx^2}=k\dfrac{(k+1)T_k-U_k}{x^2-1}$


* $\dfrac{d^2 T_k}{dx^2}_{x=1}=\dfrac{k^4-k^2}{3}$


* $\dfrac{d^2 T_k}{dx^2}_{x=-1}=(-1)^k\dfrac{k^4-k^2}{3}$

In [30]:
function d2chebyshev(x,k)
    chebyshev_Tk=Jacobi.chebyshev(x, k)
    chebyshev_Uk=Jacobi.chebyshev2(x, k)
    if x!=1&-1
        d2_Tk=k*((k+1)*chebyshev_Tk-chebyshev_Uk)/(x^2-1)
    end
    if x==1
        d2_Tk=(k^4-k^2)/3
    end
    if x==-1
        d2_Tk=(-1)^k*(k^4-k^2)/3
    end
    return d2_Tk
end

d2chebyshev (generic function with 1 method)

* basisfunc: $\prod\limits_{i=1}^N \sum\limits_{z=1}^{N} \prod\limits_{j=1}^d T_{k_{ij}}(r_{zj})* \prod\limits_{i=1}^N \prod\limits_{j=1}^d (r_c^2-r_{ij}^2)$

* Prod_Tk: $\prod\limits_{j=1}^d T_{k_{ij}}(r_{zj})$
* Sum_Tk: $\sum\limits_{z=1}^{N} \prod\limits_{j=1}^d T_{k_{ij}}(r_{zj})$

In [49]:
function laplace_psi(N,d,r,k,rc)
    Sum_laplace=0
    Prod_laplace=0
    basisfunc=basis_function(N,d,r,k,rc)[1]
    Sum_Tk=basis_function(N,d,r,k,rc)[2]
    Prod_Tk=basis_function(N,d,r,k,rc)[3]
        for n=1:d
            for m=1:N
                for i=1:N-1
                #################d2T####################
                d2T_k=d2chebyshev(r[m,n], k[i,n])
                T_k=Jacobi.chebyshev(r[m,n], k[i,n])
                Prod_laplace=basisfunc*d2T_k*Prod_Tk[i,m]/(Sum_Tk[i]*T_k)
                Sum_laplace+=Prod_laplace
                #################dTdT####################
                dT_ki=Jacobi.dchebyshev(r[m,n], k[i,n])
                T_k_i=Jacobi.chebyshev(r[m,n], k[i,n])
                    for j=i+1:N
                    dT_kj=Jacobi.dchebyshev(r[m,n], k[j,n])
                    T_k_j=Jacobi.chebyshev(r[m,n], k[j,n])
                    Prod_laplace=basisfunc*dT_kj*dT_ki*Prod_Tk[i,m]*Prod_Tk[j,m]/(Sum_Tk[i]*Sum_Tk[j]*T_k_i*T_k_j)
                    Sum_laplace+=2*Prod_laplace
                    end
                #################dTdx####################
                Prod_laplace=-2*r[m,n]*basisfunc*dT_ki*Prod_Tk[i,m]/(Sum_Tk[i]*(rc^2-r[m,n]^2)*T_k_i)
                Sum_laplace+=2*Prod_laplace
                end
            T_k=Jacobi.chebyshev(r[m,n], k[N,n])
            d2T_k=d2chebyshev(r[m,n], k[N,n])
            Prod_laplace=basisfunc*d2T_k*Prod_Tk[N,m]/(Sum_Tk[N]*T_k)
            Sum_laplace+=Prod_laplace
            T_k_i=Jacobi.chebyshev(r[m,n], k[N,n])
            dT_ki=Jacobi.dchebyshev(r[m,n], k[N,n])
            Prod_laplace=-2*basisfunc*dT_ki*r[m,n]*Prod_Tk[N,m]/(Sum_Tk[N]*(rc^2-r[m,n]^2)*T_k_i)
            Sum_laplace+=2*Prod_laplace
            #################d2x####################
            Sum_laplace+=-2*basisfunc/(rc^2-r[m,n]^2)
            end
        end
    return Sum_laplace
end

laplace_psi (generic function with 1 method)

In [32]:
function V_psi(N,d,r,k,range)
    Sum_V=0
    Sum_r=sum(r.^2)
        Sum_V=basis_function(N,d,r,k,range)[1]*Sum_r
    return Sum_V
end

V_psi (generic function with 1 method)

In [33]:
function H_psi(N,d,r,k,range)
       Sum_H_psi=-1/2*laplace_psi(N,d,r,k,range)+1/2*V_psi(N,d,r,k,range)
    return Sum_H_psi
end

H_psi (generic function with 1 method)

#### nsumk(N,M):
* divide M into N parts
* eg. N=2,M=3
* $[:,:,1]=\begin{bmatrix} 3 & 0\end{bmatrix}=\begin{bmatrix} k_{11} & k_{12}\end{bmatrix}$
where $k_{11}\geq k_{12}$ 

In [34]:
function nsumk(N,M)
    k=collect(partitions(M+N,N))
    k=vcat(map(e->collect(e)', k)...)
    k=k-ones(Int64,size(k))
    return k
end

nsumk (generic function with 1 method)

In [54]:
nsumk(2,3)

2×2 Array{Int64,2}:
 3  0
 2  1

#### find_k(d,N,M):
* divide $i=1,2,\cdots,M$ into $d*N$ parts
* eg. d=3,N=2,M=2
* $[:, :, 14] =\begin{bmatrix} 0 & 0 & 0 \\ 1 & 1 & 0\end{bmatrix}=\begin{bmatrix} k_{11} & k_{12} & k_{13} \\ k_{21} & k_{22} & k_{23}\end{bmatrix}$
* $\begin{bmatrix} k_{11} & k_{12} & k_{13}\end{bmatrix}\preceq \begin{bmatrix} k_{21} & k_{22} & k_{23}\end{bmatrix}$, where $\preceq$ denotes the lexicographical order.  

In [36]:
function find_k(d,N,M)
    k_all=zeros(Int64,1,N*d)
    for i=1:M
        dividers=nsumk(N*d,i)
        k_all=vcat(k_all, dividers)
    end
    k_all=sort!(k_all,dims =2)
    k_all=k_all'
    number_of_k=size(k_all)[2]
    i=0
    k=collect(permutations(k_all[1+i*N*d:N*d+i*N*d]))
    k=vcat(map(e->collect(e)', k)...)
    for i=1:number_of_k-1
        k_temp=collect(permutations(k_all[1+i*N*d:N*d+i*N*d]))
        k_temp=vcat(map(e->collect(e)', k_temp)...)
        k=vcat(k, k_temp)
        k=unique(k,dims=1)
    end
    k=k'
    number_of_k=size(k)[2]
    k=reshape(k,number_of_k*N*d)
    k=reshape(k,N,d,number_of_k)
    for i=1:number_of_k
        k[:,:,i]=sortslices(k[:,:,i],dims=1)
    end
    k=unique(k,dims=3)  
   return k
end

find_k (generic function with 1 method)

In [37]:
find_k(3,2,3)

2×3×44 Array{Int64,3}:
[:, :, 1] =
 0  0  0
 0  0  0

[:, :, 2] =
 0  0  0
 0  0  1

[:, :, 3] =
 0  0  0
 0  1  0

...

[:, :, 42] =
 0  1  0
 1  1  0

[:, :, 43] =
 1  0  0
 1  0  1

[:, :, 44] =
 1  0  0
 1  1  0

In [57]:
function random_choose(Number_of_x,d,N,k,rc)
    Number_of_k=size(k)[3]
    random_choose_X=rand(N,d,Number_of_x).*rand([-rc,rc],N,d,Number_of_x)
    random_choose_psi_x=zeros(Number_of_k,Number_of_x)
    random_choose_Hpsi_x=zeros(Number_of_k,Number_of_x)
    for i=1:Number_of_k
        for j=1:Number_of_x
            random_choose_psi_x[i,j]=basis_function(N,d,random_choose_X[:,:,j],k[:,:,i],range)[1]
            random_choose_Hpsi_x[i,j]=H_psi(N,d,random_choose_X[:,:,j],k[:,:,i],range)
        end
    end
    return random_choose_psi_x,random_choose_Hpsi_x,random_choose_X
end

random_choose (generic function with 1 method)

In [39]:
function least_squares(least_squares_x,least_squares_y)
    least_squares_C_k=inv(least_squares_x'*least_squares_x)*least_squares_x'*least_squares_y
    return least_squares_C_k
end

least_squares (generic function with 1 method)

#### $c_n$=$c_{n-1}-\alpha * p_{n-1}$

In [58]:
function Eigenvalue(random_x,coef,alpha,d,N,k,rc,Number_of_x;eps_x = 10^(-20),eps_d=0.001,maxIterations = 100)
    record=spzeros(maxIterations)
    basisfunction=random_x[1]
    H_basisfunction=random_x[2]
    for t in 1:maxIterations
            reilaygh_quotientalpha=reilaygh_quotient_alpha(coef,basisfunction,H_basisfunction,N,d,rc,Number_of_x)
            reilaygh_quotient=reilaygh_quotientalpha[1]
            H_wave=reilaygh_quotientalpha[2]
            coef=reilaygh_quotientalpha[3]
            p=reilaygh_quotientalpha[4]
            coef_H_wave=reilaygh_quotientalpha[5]
            record[t]=reilaygh_quotient[1]
        if t>=3
            if abs(record[t]-record[t-1])+abs(record[t-1]-record[t-2])<=2*eps_d
                println("Convergence is reached after  ",t,"  iterations.")
                return coef,reilaygh_quotient,record
            end
        end
        alpha=0.01
        #steep(p,coef,reilaygh_quotient,coef_H_wave,basisfunction,N,d,range,H_basisfunction,Number_of_x,0,0.1,0.01,100)[1]
        coef = coef-alpha*p
        println("At step ", t, " and reilaygh quotient = ", reilaygh_quotient)
    end
    println("Warning:",maxIterations,"  iterations have been exceeded")
    return coef,reilaygh_quotient,record
end

Eigenvalue (generic function with 1 method)

* reilaygh_quotient : $\frac{\sum \langle \phi | H |\phi\rangle }{\sum \langle \phi|\phi\rangle}$

In [62]:
function reilaygh_quotient_alpha(coef_temp,basisfunction,H_basisfunction,N,d,rc,Number_of_x)
        wavefunc=coef_temp*basisfunction
        sum_2_wavefunc=wavefunc*wavefunc'
        integal_wavefunc=(2*rc)^(N*d)*sum_2_wavefunc/Number_of_x
        coef_temp=coef_temp/sqrt(integal_wavefunc[1])
        wavefunc=wavefunc/sqrt(integal_wavefunc[1])
        sum_2_wavefunc=sum_2_wavefunc/integal_wavefunc[1]
    ########################################################
        H_wavefunc=coef_temp*H_basisfunction
        sum_wavefunc_Hwavefunc=wavefunc*H_wavefunc'
        reilaygh_quotient=sum_wavefunc_Hwavefunc/sum_2_wavefunc
        coef_H_wave=least_squares(basisfunction',H_wavefunc')
        coef_H_wave=coef_H_wave'
        p=coef_H_wave-reilaygh_quotient*coef_temp
    return reilaygh_quotient,H_wavefunc,coef_temp,p,coef_H_wave
end

reilaygh_quotient_alpha (generic function with 1 method)

In [64]:
d=1
N=1
M=18
@time k=find_k(d,N,M)
Number_of_x=80000
rc=5
@time random_x=random_choose(Number_of_x,d,N,k,rc)
Number_k=size(k)[3]
coef=ones(1,Number_k)
eps_x = 10^(-6)
eps_d=0.001
alpha=0.001
@time A=Eigenvalue(random_x,coef,alpha,d,N,k,rc,Number_of_x;eps_x= 10^(-30),eps_d=10^(-8),maxIterations = 100000)

  0.000241 seconds (1.21 k allocations: 104.734 KiB)
  3.198995 seconds (50.16 M allocations: 4.215 GiB, 15.79% gc time)
At step 1 and reilaygh quotient = [19.119024395874707]
At step 2 and reilaygh quotient = [16.75419995276905]
At step 3 and reilaygh quotient = [15.821621252702545]
At step 4 and reilaygh quotient = [15.100214519116651]
At step 5 and reilaygh quotient = [14.528942828493513]
At step 6 and reilaygh quotient = [14.062480364384788]
At step 7 and reilaygh quotient = [13.669113860472189]
At step 8 and reilaygh quotient = [13.327669104916005]
At step 9 and reilaygh quotient = [13.02432652243938]
At step 10 and reilaygh quotient = [12.750082652661433]
At step 11 and reilaygh quotient = [12.498967095930112]
At step 12 and reilaygh quotient = [12.266878976291508]
At step 13 and reilaygh quotient = [12.050863958025916]
At step 14 and reilaygh quotient = [11.848680521251865]
At step 15 and reilaygh quotient = [11.658546173948029]
At step 16 and reilaygh quotient = [11.47899096461

([0.02421209208064836 5.24989605803268e-6 … -2.960448924574151e-20 -6.134522186804897e-18], [0.5000038143864289],   [1     ]  =  19.119
  [2     ]  =  16.7542
  [3     ]  =  15.8216
  [4     ]  =  15.1002
  [5     ]  =  14.5289
  [6     ]  =  14.0625
  [7     ]  =  13.6691
  [8     ]  =  13.3277
  [9     ]  =  13.0243
  [10    ]  =  12.7501
            ⋮
  [829   ]  =  0.500004
  [830   ]  =  0.500004
  [831   ]  =  0.500004
  [832   ]  =  0.500004
  [833   ]  =  0.500004
  [834   ]  =  0.500004
  [835   ]  =  0.500004
  [836   ]  =  0.500004
  [837   ]  =  0.500004
  [838   ]  =  0.500004
  [839   ]  =  0.500004)

In [66]:
d=1
N=2
M=8
k=find_k(d,N,M)
rc=5
Number_k=size(k)[3]
c_k=ones(1,Number_k)
eps_x = 10^(-6)
eps_d=0.001
alpha=0.001
Number_of_x=50000
record_time=zeros(1,30)
println("=================================M=",M,"==============================================")
time = @elapsed random_x=random_choose(Number_of_x,d,N,k,rc)
record_time[1]=time
time = @elapsed dn=Eigenvalue(random_x,c_k,alpha,d,N,k,rc,Number_of_x;eps_x= 10^(-30),eps_d=10^(-7),maxIterations = 100000)
df1=dn[3]
record_time[2]=time
t=1
err=1
df3=dn[2]
println("=================================df3==============================================")
print(df3)
while err>0.001
    M=M+2
    println("=================================M=",M,"==============================================")
    k=find_k(d,N,M)
    Number_k=size(k)[3]
    c_k=ones(1,Number_k)
    time = @elapsed random_x=random_choose(Number_of_x,d,N,k,rc)
    record_time[2*t+1]=time
    time = @elapsed dn=Eigenvalue(random_x,c_k,alpha,d,N,k,rc,Number_of_x;eps_x= 10^(-30),eps_d=10^(-7),maxIterations = 100000)
    df2=dn[3]  
    err=(df3-dn[2])[1]
    println("=================================err==============================================")
    print(err)
    df3=dn[2]
    record_time[2*t+2]=time
    record=hcat(df1, df2)
    df1=record
    t=t+1
end

=================================M=8==============================================
At step 1 and reilaygh quotient = [13.56724785558929]
At step 2 and reilaygh quotient = [13.169422557767664]
At step 3 and reilaygh quotient = [12.802307232463935]
At step 4 and reilaygh quotient = [12.465290554521756]
At step 5 and reilaygh quotient = [12.15678265518079]
At step 6 and reilaygh quotient = [11.874523664858998]
At step 7 and reilaygh quotient = [11.615866075654342]
At step 8 and reilaygh quotient = [11.378009555842544]
At step 9 and reilaygh quotient = [11.15818018177674]
At step 10 and reilaygh quotient = [10.953755640144266]
At step 11 and reilaygh quotient = [10.762343565286937]
At step 12 and reilaygh quotient = [10.581822533708499]
At step 13 and reilaygh quotient = [10.41035538311931]
At step 14 and reilaygh quotient = [10.24638342430686]
At step 15 and reilaygh quotient = [10.088608494901571]
At step 16 and reilaygh quotient = [9.935968123919412]
At step 17 and reilaygh quotient = [